# ChatGLM云端包
- 十字鱼 https://www.bilibili.com/video/BV1cX4y1z73s
- 阿里云专版。想训练的话，运行3.4.后再运行2.。

## 0.查看显卡信息（可跳过）

In [ ]:
!nvidia-smi

## 1.安装依赖
- 失败时请重试。（git库有时候链接不上很正常）

In [ ]:
import os
!pip install --upgrade pip
!apt update
!apt install -y aria2
%cd /mnt/workspace
!git clone https://ghproxy.com/https://github.com/AIChat-hobbyist/ChatGLM-Lora.git
if os.path.exists('ChatGLM-Lora'):
 %cd ChatGLM-Lora
 !git pull
 !pip install -r requirements.txt
 !aria2c --console-log-level=error -c -x 16 -s 16 -j 8 -i "model.txt"

## 2.运行GLM
- 运行成功后，点击gradio网址打开GLM

In [ ]:
%cd /mnt/workspace/ChatGLM-Lora
!python webui.py --path chatglm-6b

## 3.数据转换
- 9train.json是训练文本，可替换，注意格式

In [ ]:
%cd /mnt/workspace/ChatGLM-Lora
!python cover_alpaca2jsonl.py \
    --data_path 9train.json \
    --save_path 9train.jsonl
!python tokenize_dataset_rows.py \
    --jsonl_path 9train.jsonl \
    --save_path data/alpaca \
    --max_seq_length 200

## 4.训练
- 参数可修改

In [ ]:
%cd /mnt/workspace/ChatGLM-Lora
!wandb off
!python finetune.py \
    --dataset_path data/alpaca \
    --lora_rank 32 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --max_steps 1000 \
    --save_steps 1000 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir output